Evalvuate the eror 
The objective of the eval pred is to first figure out some info about the evaluation metric. The evaluation metric Consists of two parts. Ftist is a rank based one. We rank the customers that have defulted based on the predicted probability. choose the highest .04(#1 + #0*20). 

This is equvalent to the recall level of 4% the doc states. That is, of all the customers that are going to deafult, we aim to recall at least 96% of them!! 


# Q:
 - What does a recall of 4% mean? How do we train for a recall of a 4%? 

In [4]:
import pandas as pd
from pd.params import *

In [25]:
def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

In [3]:
def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    

In [18]:
import numpy as np
import torch 
from pd.nn.model import Conv

In [21]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from bes.nn.es_module import ESModule


In [22]:
class Conv(ESModule):

    def __init__(self, hidden_dim=64, output_dim=1, ):
        super(Conv, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.conv1 = nn.Conv1d(in_channels=13, out_channels=25, kernel_size=5, )
        self.conv2 = nn.Conv1d(in_channels=25, out_channels=5, kernel_size=5, )
        self.conv3 = nn.Conv1d(in_channels=5, out_channels=1, kernel_size=3, )
        self.fc1 = nn.Linear(in_features=154, out_features=hidden_dim)
        self.fc2 = nn.Linear(in_features=hidden_dim, out_features=1)
    def forward(self, h):
        h = F.selu(self.conv1(h))
        h = F.selu(self.conv2(h))
        h = F.selu(self.conv3(h))
        h = F.selu(self.fc1(h))
        return torch.sigmoid(self.fc2(h)).squeeze(-1)

In [29]:
model_name = "conv_all"
model = Conv()
model_param = torch.load(OUTDIR+model_name)
model.load_state_dict(model_param)


<All keys matched successfully>

In [30]:
train_data = np.load(OUTDIR+"train_data_all.npy")

In [26]:
train_labels = np.load(OUTDIR+"train_labels_all.npy")

In [31]:
pred = model(torch.as_tensor(train_data, dtype=torch.float32))
m =  amex_metric(train_labels, pred.detach().numpy())

array([1., 0., 0., ..., 0., 0., 0.])

In [34]:
df = pd.DataFrame({'target': train_labels.ravel(), 'prediction': pred.detach().numpy().ravel()})
    

In [35]:
df

,target,prediction
0,1.0,0.898940
1,0.0,0.064836
2,0.0,0.002584
3,1.0,0.691589
4,0.0,0.179399
...,...,...
458908,1.0,0.759691
458909,1.0,0.932255
458910,0.0,0.002209
458911,0.0,0.196208


In [36]:
df.sort_values('prediction', ascending=False, inplace=True)


In [38]:
df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
four_pct_cutoff = int(0.04 * df['weight'].sum())
df['weight_cumsum'] = df['weight'].cumsum()
df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]

276821

In [ ]:
four_pct_cutoff = int(0.04 * df['weight'].sum())
df['weight_cumsum'] = df['weight'].cumsum()
df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        

In [45]:
df_cutoff

,target,prediction,weight,weight_cumsum
332753,1.0,0.999898,1,1
50129,1.0,0.999876,1,2
360319,1.0,0.999857,1,3
296230,1.0,0.999850,1,4
257124,1.0,0.999818,1,5
...,...,...,...,...
305887,1.0,0.715532,1,276813
431282,1.0,0.715521,1,276814
51027,1.0,0.715495,1,276815
294584,1.0,0.715479,1,276816


In [46]:
(df_cutoff['target'] == 1).sum()

74477

In [47]:
(df['target'] == 1).sum()

118828

In [48]:
df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
total_pos = (df['target'] * df['weight']).sum()
df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
df['lorentz'] = df['cum_pos_found'] / total_pos
        

In [50]:
df['lorentz']

332753    0.000008
50129     0.000017
360319    0.000025
296230    0.000034
257124    0.000042
            ...   
322621    1.000000
351048    1.000000
60904     1.000000
259191    1.000000
9590      1.000000
Name: lorentz, Length: 458913, dtype: float64

In [ ]:
df['gini'] = (df['lorentz'] - df['random']) * df['weight']
